In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
# for jupyter
%matplotlib inline 

dt = pd.read_csv('Dataset/heart.csv')
"""
font: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction
Attribute Information
    Age: age of the patient [years]
    Sex: sex of the patient [M: Male, F: Female]
    ChestPainType: chest pain type [TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]
    RestingBP: resting blood pressure [mm Hg]
    Cholesterol: serum cholesterol [mm/dl] // algumas referêcias apontam mg/dl
    FastingBS: fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]
    RestingECG: resting electrocardiogram results [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria]
    MaxHR: maximum heart rate achieved [Numeric value between 60 and 202]
    ExerciseAngina: exercise-induced angina [Y: Yes, N: No]
    Oldpeak: oldpeak = ST [Numeric value measured in depression]
    ST_Slope: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]
    HeartDisease: output class [1: heart disease, 0: Normal]
"""      

print("...")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

In [ ]:
db = pd.DataFrame()
db["ExerciseAngina"] = dt["ExerciseAngina"].apply(lambda x: 1 if x == 'Y' else 0)
"""
# 1- ST_Slope
# 2- ExerciseAngina
# 3- ChestPainType
"""
#db["ChestPainType"] = dt["ChestPainType"]
#for dor_tipo in ["TA","ATA","NAP","ASY"]:
#    db["ChestPainType_"+dor_tipo] = dt["ChestPainType"].apply(lambda x: 1 if dor_tipo == x else 0)
ChestPainTypes = ["TA","ATA","NAP","ASY"]
db["ChestPainType"] = dt["ChestPainType"].apply(lambda x: ChestPainTypes.index(x)/10)



db["MaxHR"] = dt["MaxHR"].apply(lambda x: (x-60) /(202-60))
db["Oldpeak"] = dt["Oldpeak"].apply(lambda x: (x+2.6) /(6.2+2.6))

#db["ST_Slope"] = dt["ST_Slope"]
#for st_slot_tipo in ['Up', 'Flat', 'Down']:
#    db["ST_Slope"+st_slot_tipo] = dt["ST_Slope"].apply(lambda x: 1 if dor_tipo == x else 0)
db['ST_Slope'] =  dt['ST_Slope'].apply(lambda x: ['Up', 'Flat', 'Down'].index(x)/10)


#db["RestingBP"] = dt["RestingBP"].apply(lambda x: x / 200)
#db["Age"] = dt["Age"].apply(lambda x: x / 77)
 

#db["HeartDisease"] = dt["HeartDisease"]
db.head()
# 1- ST_Slope
# 2- ExerciseAngina
# 3- ChestPainType
# 4- Oldpeak ou MaxHR

#db.Oldpeak.describe()
#db.MaxHR.describe()
db.head(20)

In [ ]:
optimizer_adam = tf.keras.optimizers.Adam(learning_rate=0.001)

optimizer_SGD = tf.keras.optimizers.SGD(
    learning_rate=0.001,
    momentum=0.1,
    nesterov=False,
    name='SGD')


optimizer_Adagrad= tf.keras.optimizers.Adagrad(
    learning_rate=0.01,
    initial_accumulator_value=0.01,
    epsilon=1e-07,
    name="Adagrad",
 
)

In [ ]:

#x_verify,x_temp,y_verify,y_temp = train_test_split(db,dt['HeartDisease'],test_size= 0.09, random_state=42)
#x_treino,x_teste,y_treino,y_teste = train_test_split(x_temp,y_temp,test_size= 0.18, random_state=328)
#seeds =[4,70, 0,21,328 ]
x_treino,x_teste,y_treino,y_teste = train_test_split(db, dt['HeartDisease'],test_size= 0.3, random_state=328)
  
    
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,
                                                         write_images=True,
        write_steps_per_second=True,
                                                     
                                                     )


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(100,activation=tf.nn.relu,input_shape=(x_treino.shape[1],)))
model.add(tf.keras.layers.Dense(120,activation=tf.nn.relu ))
model.add(tf.keras.layers.Dense(1,activation=tf.nn.sigmoid ))
model.summary()
model.compile(optimizer=optimizer_adam,
                loss='binary_crossentropy',
                 metrics=['accuracy'])


model.fit(x_treino,y_treino,batch_size=128, epochs=3000,verbose=2,
                callbacks=[tensorboard_callback]                 
                )



print(model.evaluate(x_teste,  y_teste, verbose=2))
print(model.evaluate(x_treino,  y_treino, verbose=2))
